# Workspace API Client quickstart guide

`evo.workspaces.WorkspaceAPIClient` facilitates communication with the Workspace API. The first step is to set up the Authorizer and the APIConnector required for the SDK setup. For more information, take a look at the  `quickstart.ipynb` and `oauth.ipynb` examples.

In [ ]:
from evo.aio import AioTransport
from evo.oauth import AuthorizationCodeAuthorizer, EvoScopes, OAuthConnector

# OAuth client app credentials
# See: https://developer.seequent.com/docs/guides/getting-started/apps-and-tokens
REDIRECT_URL = "http://localhost:3000/signin-callback"
CLIENT_NAME = "Your Client Name"
CLIENT_ID = "your-client-id"

transport = AioTransport(
    user_agent=CLIENT_NAME,
)

connector = OAuthConnector(
    transport=transport,
    client_id=CLIENT_ID,
)

authorizer = AuthorizationCodeAuthorizer(
    oauth_connector=connector,
    redirect_url=REDIRECT_URL,
    scopes=EvoScopes.all_evo | EvoScopes.offline_access,
)

# Login to the Evo platform.
await authorizer.login()

## WorkspaceAPIClient

Now that the `authorizer` is set up, we can set up the `WorkspaceServiceClient`. This client wraps endpoint functionality to provide a cohesive interface to the underlying API client implementation.

For initialization the client requires:
* The organization ID since all workspace service operations are currently bound by this ID.
* The hub URL to connect to the workspace service.

In [ ]:
import uuid

from evo.common import APIConnector
from evo.workspaces import WorkspaceAPIClient

# See "quickstart.ipynb" for assistance in obtaining your organization ID and hub URL
ORG_ID = uuid.UUID("<ORG_ID>")
HUB_URL = "https://your-hub-url"

workspace_client = WorkspaceAPIClient(
    connector=APIConnector(HUB_URL, transport, authorizer),
    org_id=ORG_ID,
)

service_health = await workspace_client.get_service_health()

print(f"Workspace API is {service_health.status.name.lower()}")

## Workspace operations
In this section we will go over some of the available workspace operations.

### Listing workspaces
To list the workspaces you can either use `WorkspaceAPIClient.list_workspaces()` method:

In [ ]:
from evo.common import Page
from evo.workspaces import Workspace

paginated_workspaces: Page[Workspace] = await workspace_client.list_workspaces(limit=5, offset=0)

The `list_workspaces` will return a `evo.common.Page` object that encapsulates the paginated response by the Workspace Service.
Alternatively, you can list all workspaces available to the given user by calling the `WorkspaceServiceClient.list_all_workspaces()` method:

In [ ]:
all_workspaces: list[Workspace] = await workspace_client.list_all_workspaces()

Both `list_workspaces` and `list_all_workspaces` support all the query parameters supported by the workspace service's [List Workspaces](https://developer.seequent.com/docs/api/workspaces/list-workspaces) endpoint:

In [ ]:
workspaces = await workspace_client.list_workspaces(
    limit=5,
    offset=0,
    deleted=False,
    order_by={"name": "asc"},
)

### Creating a workspace

In [ ]:
new_workspace: Workspace = await workspace_client.create_workspace(
    name="My New Workspace",
    description="This is a new workspace",
    bounding_box_coordinates=[
        (85.8287, -90.0),
        (85.8287, 69.84278),
        (-180.0, 69.84278),
        (-180.0, -90.0),
        (85.8287, -90.0),
    ],
    # A square bounding box. It is important to mention that the first and last coordinates must be the same.
    # Also, the first coordinate element must be the longitude and the second element must be the latitude.
)
print(new_workspace)

### Updating a workspace

In [ ]:
updated_workspace: Workspace = await workspace_client.update_workspace(
    workspace_id=new_workspace.id,
    name="My Updated Workspace",
    description="This is an updated workspace",
)

### Deleting a workspace

In [ ]:
await workspace_client.delete_workspace(workspace_id=new_workspace.id)

## Users operation

In this section we will go over some of the available user related operations.

### Get current user's workspace role

In [ ]:
from evo.workspaces import User, UserRole

user_role: UserRole = await workspace_client.get_current_user_role(workspace_id=new_workspace.id)

### Get list of users with access to a workspace.

In [ ]:
users: list[User] = await workspace_client.list_user_roles(workspace_id=new_workspace.id)

### Assign a user to a workspace

In [ ]:
from evo.workspaces import WorkspaceRole

new_users_role: UserRole = await workspace_client.assign_user_role(
    workspace_id=new_workspace.id, user_id=uuid.UUID("<some-user-id>"), role=WorkspaceRole.viewer
)

### Remove a user from a workspace

In [ ]:
await workspace_client.delete_user_role(
    workspace_id=new_workspace.id,
    user_id=uuid.UUID("<some-user-id>"),
)